In [3]:
# 只需要執行一次, 環境準備可能會花費一些時間, 耐心等待左手邊的 [] 內從 * 變成數字
!pip install numpy pandas statsmodels
import numpy as np
import pandas as pd
import re
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [4]:
# 規格最小值最大值放進來
lcl = 32
ucl = 48

# 從 excel 將原始數據直接複製貼上, 不用額外處理
input_str = """
46.09	46.09	46.1	46.11	46.17	46.06	46.07	46.09	46.14
41.13	40.98	40.91	40.82	40.86	40.86	40.89	40.87	40.83
46.67	46.2	46.19	45.96	46.28	46.74	45.91	46.01	46.22
43.26	43.37	43.48	43.39	42.69	43.37	43.35	43.38	43.34
40.36	40.3	40.34	40.47	40.29	40.29	40.2	40.22	40.35
41.07	41.26	41.29	41.21	41.25	41.23	41.16	41.25	41.15
40.69	40.66	40.86	40.62	40.75	40.54	40.09	40.34	40.61
43	42.93	42.92	42.84	42.96	42.82	42.83	42.87	42.81
43.2	43.17	43.14	43.17	42.99	43.22	43.04	43.12	43.1
39.8	39.26	39.6	39.47	39.78	39.36	39.43	39.47	39.44
"""

In [4]:
# 前面修改完後執行該格看結果

tolerance = ucl - lcl

def parse_squashed_floats(input_str):
    data = []
    float_pattern = re.compile(r'-?\d+(?:\.\d+)?')  # Match float numbers
    for line in input_str.strip().split('\n'):
        floats = list(map(float, float_pattern.findall(line)))
        data.append(floats)
    return data
    
input_list = parse_squashed_floats(input_str)
raw_data = np.array(input_list)
raw_data = np.transpose(raw_data)

n_trials = 3
operator_ids = list(range(1, 1+len(raw_data)//n_trials))
part_ids = list(range(1, 1+len(raw_data[0])))
records = []

for op_index in range(len(operator_ids)):
    for trial in range(n_trials):
        row = raw_data[op_index * n_trials + trial]
        for part_idx, measurement in enumerate(row):
            records.append({
                'Operator': operator_ids[op_index],
                'Trial': trial + 1,
                'Part': f'Part{part_idx+1}',
                'Measurement': measurement
            })

df = pd.DataFrame(records)

# Perform Two-Way ANOVA without interaction
model = ols('Measurement ~ C(Part) + C(Operator)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=1)
anova_table['mean_sq'] = anova_table['sum_sq'] / anova_table['df']

# Extract Mean Squares
MS_part = anova_table.loc['C(Part)', 'mean_sq']
MS_operator = anova_table.loc['C(Operator)', 'mean_sq']
MS_repeat = anova_table.loc['Residual', 'mean_sq']

n_operators = df['Operator'].nunique()
n_parts = df['Part'].nunique()

# Variance Components
var_repeat = MS_repeat
var_operator = max((MS_operator - MS_repeat) / (n_parts * n_trials), 0)
var_part = max((MS_part - MS_repeat) / (n_operators * n_trials), 0)
var_grr = var_repeat + var_operator
var_total = var_repeat + var_operator + var_part

# Standard deviations
sd_repeat = np.sqrt(var_repeat)
sd_operator = np.sqrt(var_operator)
sd_part = np.sqrt(var_part)
sd_grr = np.sqrt(var_grr)
sd_total = np.sqrt(var_total)

results = {
    'Source (來源)': ['Total Gage R&R', 'Repeatability', 'Reproducibility', 'Part-to-Part', 'Total Variation'],
    'SD (標準差)': [sd_grr, sd_repeat, sd_operator, sd_part, sd_total],
    '%Study Variation (%研究變異)': [6 * sd_grr, 6 * sd_repeat, 6 * sd_operator, 6 * sd_part, 6 * sd_total] / (6 * sd_total) * 100,
    '%Tolerance (%公差)': [
       100 * 6 * sd_grr / tolerance,
       100 * 6 * sd_repeat / tolerance,
       100 * 6 * sd_operator / tolerance,
       100 * 6 * sd_part / tolerance,
       100 * 6 * sd_total / tolerance]
}

result_df = pd.DataFrame(results)
pd.set_option('display.float_format', '{:.4f}'.format)
print("\nGage R&R Summary:")
print(result_df)

# NDC = 1.41 × (Part SD / GRR SD)
ndc = 1.41 * sd_part / sd_grr
print(f"\n可區分的類別數 (NDC): {ndc:.2f}, 向下取整數 {int(ndc)}.")

<class 'NameError'>: name 're' is not defined